In [1]:
import context
from rateslib import *
import pandas as pd
import datetime
import time

In [2]:
start = time.time()

# Load IRS from CSV

In [3]:
# df = pd.read_csv("100swaps.csv", parse_dates=["effective", "termination"])
df = pd.read_csv("100swaps.csv")
df["effective"] = df["effective"].map(lambda x: dt.strptime(x, "%d-%b-%Y"))
df["termination"] = df["termination"].map(lambda x: dt.strptime(x, "%d-%b-%Y"))

In [4]:
start2 = time.time()
swaps = [
        IRS(
            effective=row.effective, 
            termination=row.termination, 
            frequency=row.frequency, 
            currency=row.currency,
            notional=row.notional,
            stub="SHORTBACK",
            calendar="bus",
            curves=row.currency,
            fixed_rate=row.fixed_rate,
        )
        for row in df.itertuples()
    ]
print(f"Loading swaps took {time.time() - start2}secs")

Loading swaps took 87.59821081161499secs


# Construct some Curves

In [5]:
kwargs = dict(
    effective=dt(2023, 7, 21),
    frequency="A",
)
chf_instruments = [
    IRS(termination=f"{i}Y", currency="chf", curves="chf", **kwargs) for i in range(1, 51)
]
gbp_instruments = [
    IRS(termination=f"{i}Y", currency="gbp", curves="gbp", **kwargs) for i in range(1, 51)
]
usd_instruments = [
    IRS(termination=f"{i}Y", currency="usd", curves="usd", **kwargs) for i in range(1, 51)
]
rates = [(3.0 - i/100) for i in range(50)]
chf = Curve({add_tenor(dt(2023, 7, 21), f"{i}Y", "None"): 1.0 for i in range(51)}, id="chf")
gbp = Curve({add_tenor(dt(2023, 7, 21), f"{i}Y", "F", calendar="bus"): 1.0 for i in range(51)}, id="gbp")
usd = Curve({add_tenor(dt(2023, 7, 21), f"{i}Y", "F", calendar="bus"): 1.0 for i in range(51)}, id="usd")
defaults.algorithm = "levenberg_marquardt"
solver1 = Solver(curves=[chf], instruments=chf_instruments, s=rates, id="CHF")
solver2 = Solver(curves=[gbp], instruments=gbp_instruments, s=rates, id="GBP")
solver3 = Solver(curves=[usd], instruments=usd_instruments, s=rates, id="USD", pre_solvers=[solver1, solver2])

SUCCESS: `func_tol` reached after 7 iterations (levenberg_marquardt) , `f_val`: 7.611210062589067e-13, `time`: 1.9036s
SUCCESS: `func_tol` reached after 7 iterations (levenberg_marquardt) , `f_val`: 7.635491507221569e-13, `time`: 1.8914s
SUCCESS: `func_tol` reached after 7 iterations (levenberg_marquardt) , `f_val`: 7.635491507221569e-13, `time`: 1.8670s


In [6]:
pf = Portfolio(swaps)

In [7]:
defaults.cores = 5

In [8]:
# for i, swap in enumerate(swaps):
#     print(i)
#     swap.npv(solver=solver3)

In [9]:
npv = pf.npv(solver=solver3, local=True)

In [10]:
solver3.delta(npv)

local_ccy                           chf  gbp            usd
display_ccy                         chf  gbp            usd
type        solver label                                   
instruments CHF    CHF0    15526.140051  0.0       0.000000
                   CHF1    44775.652947  0.0       0.000000
                   CHF2    68209.460854  0.0       0.000000
                   CHF3   550873.214572  0.0       0.000000
                   CHF4   225472.506439  0.0       0.000000
...                                 ...  ...            ...
            USD    USD45       0.000000  0.0  214378.868528
                   USD46       0.000000  0.0  825604.617603
                   USD47       0.000000  0.0  -64858.936069
                   USD48       0.000000  0.0  549729.860501
                   USD49       0.000000  0.0 -821216.500268

[150 rows x 3 columns]

In [11]:
end = time.time()

In [12]:
print(end-start)

137.78131794929504
